In [6]:
import pandas as pd
import sqlite3

In [7]:
conn = sqlite3.connect('sqlite/test.db')

In [9]:
c = conn.cursor()

# Create table
c.execute('''CREATE TABLE professors (
firstname text,
lastname text
);''')

# Add a column to the professors table
c.execute('''ALTER TABLE professors
ADD COLUMN university_shortname text;''')

# Save (commit) the changes
conn.commit()

# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
conn.close()

In [10]:
conn = sqlite3.connect('sqlite/test.db')

In [11]:
c = conn.cursor()

# Create table
c.execute('''CREATE TABLE affiliations (
organisation text);
''')

# Rename a column
c.execute('''ALTER TABLE affiliations
RENAME COLUMN organisation TO organization;''')

# Save (commit) the changes
conn.commit()

In [12]:
columns = ['firstname', 
           'lastname', 
           'university', 
           'university_shortname', 
           'university_city', 
           'function', 
           'organization', 
           'organization_sector']

uni_prof_df = pd.read_csv('uni_prof.csv', names=columns)

uni_prof_df.head()

,firstname,lastname,university,university_shortname,university_city,function,organization,organization_sector
0,Karl,Aberer,ETH Lausanne,EPF,Lausanne,Chairman of L3S Advisory Board,L3S Advisory Board,Education & research
1,Karl,Aberer,ETH Lausanne,EPF,Lausanne,Member Conseil of Zeno-Karl Schindler Foundation,Zeno-Karl Schindler Foundation,Education & research
2,Karl,Aberer,ETH Lausanne,EPF,Lausanne,Member of Conseil Fondation IDIAP,Fondation IDIAP,Education & research
3,Karl,Aberer,ETH Lausanne,EPF,Lausanne,Panel Member,SNF Ambizione Program,Education & research
4,Reza Shokrollah,Abhari,ETH Zürich,ETH,Zurich,Aufsichtsratsmandat,PNE Wind AG,Energy


In [13]:
uni_prof_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 8 columns):
firstname               100 non-null object
lastname                100 non-null object
university              100 non-null object
university_shortname    100 non-null object
university_city         100 non-null object
function                100 non-null object
organization            100 non-null object
organization_sector     100 non-null object
dtypes: object(8)
memory usage: 6.3+ KB


In [ ]:
# Create table
c.execute('''CREATE TABLE university_professors (
firstname text,
lastname text,
university text,
university_shortname text,
university_city text,
function text,
organization text,
organization_sector text
);''')

# Save (commit) the changes
conn.commit()

In [17]:
# Insert values from uni_prof_df DataFrame into table
c.executemany('INSERT INTO university_professors VALUES (?,?,?,?,?,?,?,?)', 
              uni_prof_df.values.tolist())

# Save (commit) the changes
conn.commit()

In [18]:
c.execute('''-- Insert unique professors into the new table
INSERT INTO professors 
SELECT DISTINCT firstname, lastname, university_shortname 
FROM university_professors;''')

conn.commit()

In [20]:
for row in c.execute('SELECT * FROM professors;'):
    print(row)

('Karl', ' Aberer', ' EPF')
('Reza Shokrollah', ' Abhari', ' ETH')
('Georges', ' Abou Jaoudé', ' EPF')
('Hugues', ' Abriel', ' UBE')
('Daniel', ' Aebersold', ' UBE')
('Christoph', ' Aebi', ' UBE')
('Marcelo', ' Aebi', ' ULA')
('Patrick', ' Aebischer', ' EPF')
('Stephan', ' Aier', ' USG')
('Anastasia', ' Ailamaki', ' EPF')
('Cesla', ' Amarelle', ' UNE')
('Manuel', ' Ammann', ' USG')
('Christophe', ' Ancey', ' EPF')
('Marilyne', ' Andersen', ' EPF')
('Jean-Marie', ' Annoni', ' UFR')
('John', ' Antonakis', ' ULA')
('Omid', ' Aschari', ' USG')
('David', ' Atienza Alonso', ' EPF')
('Jean-Jacques', ' Aubert', ' UNE')
('Drahomir', ' Aujesky', ' UBE')
('Johan', ' Auwerx', ' EPF')
('François', ' Avellan', ' EPF')
('Kay Werner', ' Axhausen', ' ETH')
('Philippe', ' Bacchetta', ' ULA')
('Andrea', ' Back', ' USG')
('Marco', ' Bakker', ' EPF')
('Artur', ' Baldauf', ' UBE')
('Christophe', ' Ballif', ' EPF')
('Yann', ' Barrandon', ' ULA')
('David Andrew', ' Barry', ' EPF')
('Pius Eliseo', ' Baschera',

In [24]:
# Rename a column
c.execute('''ALTER TABLE affiliations
ADD COLUMN firstname text;''')

c.execute('''ALTER TABLE affiliations
ADD COLUMN lastname text;''')

c.execute('''ALTER TABLE affiliations
ADD COLUMN function text;''')

# Save (commit) the changes
conn.commit()

In [28]:
for row in c.execute('''PRAGMA table_info('affiliations')'''):
    print(row)

(0, 'organization', 'text', 0, None, 0)
(1, 'COLUMNS', 'firstname text', 0, None, 0)
(2, 'firstname', 'text', 0, None, 0)
(3, 'lastname', 'text', 0, None, 0)
(4, 'function', 'text', 0, None, 0)


In [33]:
# Create table because of 'COLUMNS' column error
c.execute('''CREATE TABLE affiliations1 (
firstname text,
lastname text,
function text,
organisation text);
''')

conn.commit()

In [34]:
c.execute('''DROP TABLE affiliations;''')

conn.commit()

In [36]:
c.execute('''ALTER TABLE affiliations1
RENAME TO affiliations;''')

conn.commit()

In [37]:
for row in c.execute('''PRAGMA table_info('affiliations')'''):
    print(row)

(0, 'firstname', 'text', 0, None, 0)
(1, 'lastname', 'text', 0, None, 0)
(2, 'function', 'text', 0, None, 0)
(3, 'organisation', 'text', 0, None, 0)


In [38]:
c.execute('''INSERT INTO affiliations 
SELECT DISTINCT firstname, lastname, function, organization 
FROM university_professors;''')

conn.commit()

In [42]:
res = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
for name in res:
    print(name[0])

professors
university_professors
affiliations


In [43]:
c.execute('''CREATE TABLE organizations (
organization text,
organization_sector text);''')

c.execute('''INSERT INTO organizations
SELECT DISTINCT organization, organization_sector
FROM university_professors;''')

conn.commit()

In [44]:
c.execute('''CREATE TABLE universities (
university_shortname text,
university text,
university_city text);''')

c.execute('''INSERT INTO universities
SELECT DISTINCT university_shortname, university, university_city
FROM university_professors;''')

conn.commit()

In [45]:
c.execute('DROP TABLE university_professors;')

conn.commit()

In [46]:
res = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
for name in res:
    print(name[0])

professors
affiliations
organizations
universities


In [48]:
for row in c.execute('''SELECT DISTINCT(university_shortname) 
FROM professors;'''):
    print(row[0])

 EPF
 ETH
 UBE
 ULA
 USG
 UNE
 UFR


OperationalError: near "CONSTRAINT": syntax error